## TRANSFORMS

In [16]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [17]:
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=troch.float).scatter_(0, torch.tensor(y),
    value=1))
)

In [18]:
# ToTensor 은 PIL Image나 NumPy ndarray 를 FloatTensor 로 변환하고, Intensity(이미지 픽셀의 크기) 값을 [0., 1.] 범위로 비례하여 scale합니다.

# Lambda Transform 은 사용자 정의 lambda 함수를 적용합니다. 여기에서는 정수를 one-hot으로 부호화된 텐서로 바꾸는 함수를 정의합니다. 
# 이 함수는 먼저 (데이터셋 정답의 개수인) 크기 10짜리 zero tensor을 만들고, scatter_ 를 호출하여 주어진 정답 y 에 해당하는 인덱스에 value=1 을 할당합니다.

target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

## BUILD THE NEURAL NETWORK

In [33]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [35]:
# Define the Class

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [36]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [37]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


In [38]:
# Model Layers

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [41]:
# nn.Flaten

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [42]:
# nn.Linear

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [43]:
# nn.ReLU

relu = nn.ReLU()
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = relu(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[ 0.0488,  0.3249, -0.0247, -0.2240,  0.2316,  0.4888,  0.1591,  0.0431,
          0.3142,  0.3149, -0.6614,  0.0369,  0.4973, -0.4354, -0.4046, -0.1649,
         -0.4264, -0.0483, -0.7611,  0.2457],
        [ 0.2215,  0.2788, -0.0804, -0.1715,  0.1581,  0.3647,  0.2232, -0.3929,
          0.5055,  0.0333, -0.8748, -0.2993,  0.6857, -0.2859, -0.1222, -0.1870,
          0.1875,  0.1448, -0.4415,  0.7327],
        [ 0.0217,  0.2126,  0.3172, -0.0883, -0.0766,  0.2099,  0.1806, -0.3053,
          0.4179, -0.1914, -0.5641,  0.1521,  0.4418, -0.2827, -0.5237, -0.2238,
         -0.2590,  0.1822, -0.5627,  0.7322]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0488, 0.3249, 0.0000, 0.0000, 0.2316, 0.4888, 0.1591, 0.0431, 0.3142,
         0.3149, 0.0000, 0.0369, 0.4973, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2457],
        [0.2215, 0.2788, 0.0000, 0.0000, 0.1581, 0.3647, 0.2232, 0.0000, 0.5055,
         0.0333, 0.0000, 0.0000, 0.6857, 0.0000, 0.000

In [48]:
# nn.Sequential

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [49]:
# nn.Softmax

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [50]:
# Model Parameters

print("Model structure: ", model, "\n\n")

for name, param, in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0002, -0.0325,  0.0195,  ...,  0.0333,  0.0251,  0.0059],
        [ 0.0159, -0.0342, -0.0005,  ...,  0.0346,  0.0253, -0.0074]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0193, 0.0258], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0396, -0.0040, -0.0438,  ..., -0.0010,  0.0279,  0.0377],
        [ 0.0196, -0.0109,  0.0220,  ...,  0.0333, -0.0144, -0.0003]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack